In [2]:
import torch
import torch.nn.functional as F
from torch import nn
import typing

/home/sung/anaconda3/envs/cose362/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 32
seq_size = 20
num_classes = 31

In [53]:
import numpy as np
a = np.array(["O", "R", "B-", "O"])
O_idx = np.where(a=="O")[0]
drop_idx = np.random.choice(O_idx, size = int(np.round(0.3 * len(O_idx))))
select_idx = np.delete(np.arange(len(a)), drop_idx)
print(select_idx)
a = a[select_idx]
b = [1, 2, 3, 4, 5]
b = [b[i] for i in select_idx]
b

[1 2 3]


[2, 3, 4]

In [21]:
a = torch.tensor([1, 2, 3, 4])
print(torch.sum((a>=1) & (a<=3)).item())

3


In [15]:
def _one_hot(labels: torch.Tensor, num_classes) -> torch.LongTensor:
    return F.one_hot(labels, num_classes=num_classes) # One-hot vector

def _one_hot_2(n, k):
    tensor = torch.zeros(n, dtype=torch.long)
    tensor[k] = 1
    return tensor

In [4]:
ans = torch.rand(size = (3, 4, 5))
ans

tensor([[[0.6757, 0.0224, 0.2347, 0.2696, 0.5167],
         [0.8076, 0.4949, 0.2992, 0.3418, 0.6117],
         [0.0904, 0.4090, 0.3990, 0.7726, 0.1875],
         [0.5338, 0.8050, 0.7884, 0.6412, 0.0589]]])

In [5]:
test = torch.randint(5, size = (3, 4))
u_test = test.unsqueeze(-1)
print(test, u_test.size())

tensor([[3, 1, 4, 3]]) torch.Size([1, 4, 1])


In [6]:
out_1 = torch.gather(ans, dim = -1, index = u_test)
out_2 = (ans * (_one_hot(test, 5))).sum(-1).unsqueeze(-1)
print(torch.equal(out_1, out_2), out_1.size(), out_2.size(), sep = '\n')
out = out_1

True
torch.Size([1, 4, 1])
torch.Size([1, 4, 1])


In [7]:
margins = torch.clamp(ans - out + 1.0, min=0)

In [8]:
margins_1 = margins.scatter(-1, u_test, 0)
margins_2 = margins * (1 - _one_hot(test, 5))
output = margins_1
print(margins_1.size(), margins_2.size(), torch.equal(margins_1, margins_2), sep = '\n')

torch.Size([1, 4, 5])
torch.Size([1, 4, 5])
True


In [16]:
res1 = torch.stack([_one_hot_2(5, i) for i in torch.argmax(output, dim=-1)])
res2 = _one_hot(torch.argmax(output, dim=-1), 5)
print(torch.argmax(output, dim=-1), res1, res2, torch.equal(res1, res2), sep='\n')

tensor([0, 0, 3, 1])
tensor([[1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0]])
tensor([[1, 0, 0, 0, 0],
        [1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0]])
True


In [10]:
labels = torch.randint(num_classes, size = (batch_size, seq_size))
print(labels.size(), labels, labels.sum(-1), sep='\n')

torch.Size([32, 20])
tensor([[30, 15,  6, 21, 29,  6,  4,  2, 21,  2, 15, 26,  3, 15, 16, 28, 23,  7,
         25,  7],
        [28, 12, 27, 29, 17,  6,  2, 27, 11, 30,  3, 15, 18, 19,  0,  2, 12,  6,
         16,  1],
        [25, 16, 17,  2, 23, 14, 24, 23, 15,  0,  8, 13,  7, 21, 29,  7, 20,  6,
          3, 11],
        [ 3, 22,  5, 23, 29,  0, 16,  8, 18, 20, 11, 29,  9, 13, 12, 18, 28, 14,
         15, 22],
        [24, 26,  7, 10, 29, 19, 19, 11, 14, 26,  6, 12,  5, 19, 28, 27,  6, 29,
         14, 24],
        [ 2,  5, 19, 29, 26, 15, 18, 10,  4, 21, 28,  6, 10, 14, 14,  7, 11,  7,
         30, 13],
        [18,  2, 28, 19,  8, 29, 25, 26, 21,  0,  0, 17, 20,  2,  8, 26,  3, 16,
          9, 14],
        [ 8,  2,  9, 13, 21, 21, 28, 12,  4,  3,  5, 11, 12, 13, 25,  6, 28,  7,
         17,  6],
        [19, 18, 16,  5,  4, 29,  5,  4,  2,  8,  3, 23,  5, 15, 12,  4, 16, 27,
          3, 18],
        [13, 21, 16,  7,  0, 30,  8, 15, 22, 28, 15, 23,  5, 27, 22, 22, 19,  8,
       

In [11]:
res = F.one_hot(labels, num_classes= num_classes)
print(res.size(), res, sep='\n')

torch.Size([32, 20, 31])
tensor([[[0, 0, 0,  ..., 0, 0, 1],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        ...,

        [[0, 0, 0,  ..., 1, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 1,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 1, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]],

        [[0, 0, 0,  ..., 0, 1, 0],
         [0, 0, 0,  ..

In [12]:
# # vector x vector
# tensor1 = torch.randn(3)
# tensor2 = torch.randn(3)
# torch.matmul(tensor1, tensor2).size()
# # matrix x vector
# tensor1 = torch.randn(3, 4)
# tensor2 = torch.randn(4)
# torch.matmul(tensor1, tensor2).size()
# # batched matrix x broadcasted vector
# tensor1 = torch.randn(10, 3, 4)
# tensor2 = torch.randn(4)
# torch.matmul(tensor1, tensor2).size()
# batched matrix x batched matrix
# tensor1 = torch.randn(10, 3, 4)
# tensor2 = torch.randn(10, 4, 5)
# (tensor1@tensor2).size()
# batched matrix x broadcasted matrix
tensor1 = torch.randn(10, 3, 4)
tensor2 = torch.randn(4, 5)
print(torch.matmul(tensor1, tensor2).size(), (tensor1@tensor2).size(), torch.equal(torch.matmul(tensor1, tensor2), tensor1@tensor2))

torch.Size([10, 3, 5]) torch.Size([10, 3, 5]) True
